<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Analyzing Mexican Restaurants in Rosedale</font></h1>


## Introduction

For the Final Capstone of the IBM Data Science Course, there is a group of investors taht want to open a new Mexican Restaurant in Toronto, Canada. I have made a previous analysis about the Neighborhoods in Toronto, and the stakeholders decided to review 3 Neighborhoods:Lawrence Park, North Toronto and Rosadale. 
This Notebook will focus on reviewing **Rosedale Neiborhood**. I will use other Notebooks to analyze the other 2 Neighborhoods.


## Table of Contents

1.  <a href="#item1">Using Foursquare to find Mexican Restaurants Nearby</a>
2.  <a href="#item2">Explore a Given Mexican Restaurant Venue</a>  
3.  <a href="#item3">Explore Venyes in the loacation</a>  
4.  <a href="#item4">Get Trending Venues</a>  



### Import necessary Libraries


In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Define Foursquare Credentials and Version


##### Make sure that you have created a Foursquare developer account and have your credentials handy


In [2]:
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
ACCESS_TOKEN = 'your FourSquare Access Token' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100
print('Your credentails are saved')


Your credentails are saved


#### Let's Analyze Rosedale Neighborhood


We will use the coordinates from our previous lab for this Neighborhood.


### Rosedale Location 

In [3]:
latitude = 43.6795626
longitude = -79.3775294

In [4]:
import json

<a id="item1"></a>


## 1. Search for a specific venue category

> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**


#### Now, let's assume that it is lunch time, and you are craving Mexican food. So, let's define a query to search for Mexican food that is within 1000 meters from the Location.


In [ ]:
search_query = 'Mexican'
radius = 1050
print(search_query + ' .... OK!')

#### Define the corresponding URL


In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)


#### Send the GET Request and examine the results


In [6]:
import json
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '600a0f5df1d69f48c7a8a87b'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 11}}],
 'response': {'venues': [{'id': '5749aa1c498eeaadc4f2ea7c',
    'name': 'El Rinconcito Mexicano',
    'location': {'address': 'Church 636',
     'lat': 43.66984,
     'lng': -79.38282,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.66984,
       'lng': -79.38282}],
     'distance': 1163,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['Church 636', 'Toronto ON', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d153941735',
      'name': 'Burrito Place',
      'pluralName': 'Burrito Places',
      'shortName': 'Burritos',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/burrito_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1611272029',
    'hasPerk': False}]}}

In [8]:
if len(results['response']['venues']) == 0:
    print('There are not Mexican Restaurants')
    MexicanRestaurants = False
else:
    print('There are {} Mexican Restaurants around Lawrence Park.'.format(len(results['response']['venues'])))
    MexicanRestaurants = True

There are 1 Mexican Restaurants around Lawrence Park.


## Save request only if there are Mexican Restaurants as a JSON File

In [27]:
with open('MexicanRestaurantsRosedale.json', 'w') as f:
    json.dump(results, f)

#### Get relevant part of JSON and transform it into a _pandas_ dataframe


In [9]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress
0,5749aa1c498eeaadc4f2ea7c,El Rinconcito Mexicano,"[{'id': '4bf58dd8d48988d153941735', 'name': 'B...",v-1611272029,False,Church 636,43.66984,-79.38282,"[{'label': 'display', 'lat': 43.66984, 'lng': ...",1163,CA,Toronto,ON,Canada,"[Church 636, Toronto ON, Canada]"


#### Define information of interest and filter dataframe


In [10]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]


In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [11]:
# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,id
0,El Rinconcito Mexicano,Burrito Place,Church 636,43.66984,-79.38282,"[{'label': 'display', 'lat': 43.66984, 'lng': ...",1163,CA,Toronto,ON,Canada,"[Church 636, Toronto ON, Canada]",5749aa1c498eeaadc4f2ea7c


## Save Mexican Restaurants to CSV File

In [12]:
dataframe_filtered.to_csv('MexicanRosedale2.csv', index=False) 

#### Let's visualize the restaurants that are nearby


In [13]:
dataframe_filtered.name

0    El Rinconcito Mexicano
Name: name, dtype: object

In [14]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Rosedale',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

<a ><img src = "https://github.com/kmiralles/Captstone-Data-Science/blob/main/RosedaleMexRes.png?raw=true" > </a>


<a id="item2"></a>


## 2. Explore a Given Venue

> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**


### A. Let's explore the closest Mexican restaurant -- _El Rinconcito Mexicano_


In [15]:
venue_id = '5749aa1c498eeaadc4f2ea7c' # ID of El Riconcito Mexicano
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)


#### Send GET request for result


In [16]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'like', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '5749aa1c498eeaadc4f2ea7c',
 'name': 'El Rinconcito Mexicano',
 'contact': {},
 'location': {'address': 'Church 636',
  'lat': 43.66984,
  'lng': -79.38282,
  'labeledLatLngs': [{'label': 'display', 'lat': 43.66984, 'lng': -79.38282}],
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['Church 636', 'Toronto ON', 'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/el-rinconcito-mexicano/5749aa1c498eeaadc4f2ea7c',
 'categories': [{'id': '4bf58dd8d48988d153941735',
   'name': 'Burrito Place',
   'pluralName': 'Burrito Places',
   'shortName': 'Burritos',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/burrito_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 1},
 'price': {'tier': 1, 'message': 'Cheap', 'currency': '$'},
 'likes': {'count': 3,
  'groups': [{'type': 'others',
    'count': 3,
    'items': [{'id': '1512355',
      'firstName': 'Darwin',
      'lastName': 'Pal

## Save Requesto to JSON File

In [17]:
with open('AMexicanRosedale2.json', 'w') as f:
    json.dump(result, f)

### B. Get the venue's overall rating


In [18]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


That is not a very good rating. Let's check the rating of the second closest Italian restaurant.


In [ ]:
venue_id = '4f3232e219836c91c7bfde94' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

Since this restaurant has no ratings, let's check the third restaurant.


In [ ]:
venue_id = '3fd66200f964a520f4e41ee3' # ID of Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

Since this restaurant has a slightly better rating, let's explore it further.


### C. Get the number of tips


In [ ]:
result['response']['venue']['tips']['count']

### D. Get the venue's tips

> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`/tips?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**`&limit=`**LIMIT**


#### Create URL and send GET request. Make sure to set limit to get all tips


In [ ]:
## Ecco Tips
limit = 30 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&oauth_token={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION, limit)

results = requests.get(url).json()
results

#### Get tips and list of associated features


In [ ]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

#### Format column width and display all tips


In [ ]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered.reindex()

Now remember that because we are using a personal developer account, then we can access only 2 of the restaurant's tips, instead of all 15 tips.


<a id="item3"></a>


## 3. Explore a location

> `https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&limit=`**LIMIT**


#### So, we just finished reviewing Mexican Restaurants, let's explore what Venues are 1000 meters away.


In [19]:
latitude = 43.6795626
longitude = -79.3775294

#### Define URL


In [41]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)


#### Send GET request and examine results


In [42]:
import requests

In [15]:
results = requests.get(url).json()
'There are {} around Rosedale.'.format(len(results['response']['groups'][0]['items']))

'There are 21 around Rosedale.'

#### Get relevant part of JSON


In [16]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4adcb343f964a520e32e21e3',
  'name': 'Summerhill Market',
  'location': {'address': '446 Summerhill Ave',
   'crossStreet': 'btwn. MacLennan Ave. and Glen Rd.',
   'lat': 43.68626482142425,
   'lng': -79.37545823237794,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.68626482142425,
     'lng': -79.37545823237794}],
   'distance': 764,
   'postalCode': 'M4W 2E4',
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['446 Summerhill Ave (btwn. MacLennan Ave. and Glen Rd.)',
    'Toronto ON M4W 2E4',
    'Canada']},
  'categories': [{'id': '4bf58dd8d48988d118951735',
    'name': 'Grocery Store',
    'pluralName': 'Grocery Stores',
    'shortName': 'Grocery Store',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
     'suffix': '.png'},

#### Process JSON and convert it to a clean dataframe


In [17]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Summerhill Market,Grocery Store,446 Summerhill Ave,btwn. MacLennan Ave. and Glen Rd.,43.686265,-79.375458,"[{'label': 'display', 'lat': 43.68626482142425...",764,M4W 2E4,CA,Toronto,ON,Canada,[446 Summerhill Ave (btwn. MacLennan Ave. and ...,NaN,4adcb343f964a520e32e21e3
1,Black Camel,BBQ Joint,4 Crescent Rd,Yonge,43.677016,-79.389367,"[{'label': 'display', 'lat': 43.67701587839681...",994,M4W 1S9,CA,Toronto,ON,Canada,"[4 Crescent Rd (Yonge), Toronto ON M4W 1S9, Ca...",NaN,4ad88fa9f964a520511221e3
2,Toronto Lawn Tennis Club,Athletics & Sports,44 Price St.,at Yonge St.,43.680667,-79.388559,"[{'label': 'display', 'lat': 43.68066694338894...",896,NaN,CA,Toronto,ON,Canada,"[44 Price St. (at Yonge St.), Toronto ON, Canada]",NaN,4ba5156cf964a520b6da38e3
3,Craigleigh Gardens,Park,160 South Drive,at Elm Ave,43.678099,-79.371586,"[{'label': 'display', 'lat': 43.67809940868806...",505,NaN,CA,Toronto,ON,Canada,"[160 South Drive (at Elm Ave), Toronto ON, Can...",NaN,4c0960fb009a0f472fabe7bf
4,Pie Squared,Pie Shop,366 Bloor St E,Sherbourne St.,43.672143,-79.377856,"[{'label': 'display', 'lat': 43.67214294983148...",826,M4W 3R4,CA,Toronto,ON,Canada,"[366 Bloor St E (Sherbourne St.), Toronto ON M...",Downtown Toronto,54049701498ede7ef8e77dfe
5,Maison Selby,Bistro,592 Sherbourne St,Selby St,43.671232,-79.376618,"[{'label': 'display', 'lat': 43.6712319213305,...",930,M4X 1L4,CA,Toronto,ON,Canada,"[592 Sherbourne St (Selby St), Toronto ON M4X ...",NaN,5c6964c1a87921002c41f5f5
6,Starbucks,Coffee Shop,620 Jarvis St,Bloor Street,43.671082,-79.380756,"[{'label': 'display', 'lat': 43.671082, 'lng':...",979,M4Y 2H9,CA,Toronto,ON,Canada,"[620 Jarvis St (Bloor Street), Toronto ON M4Y ...",NaN,4b8fca29f964a5206e6233e3
7,Tinuno,Filipino Restaurant,31 Howard St,btwn Bleecker & Ontario St,43.671281,-79.374920,"[{'label': 'display', 'lat': 43.67128060282017...",945,M4X 1J6,CA,Toronto,ON,Canada,"[31 Howard St (btwn Bleecker & Ontario St), To...",NaN,5634ddf3498eae120169005c
8,No Frills,Grocery Store,345 Bloor St E Unit 1E,Huntley & Bloor St E,43.671616,-79.378187,"[{'label': 'display', 'lat': 43.67161641242237...",886,M4W 3J6,CA,Toronto,ON,Canada,[345 Bloor St E Unit 1E (Huntley & Bloor St E)...,Downtown Toronto,5ad0e55fad17896a74bf1177
9,Booster Juice,Juice Bar,"345 Bloor Street East, Unit #7",Huntley St,43.671566,-79.378581,"[{'label': 'display', 'lat': 43.67156553, 'lng...",894,M4W 3J6,CA,Toronto,ON,Canada,"[345 Bloor Street East, Unit #7 (Huntley St), ...",NaN,5192c223abd80089335b428c


## Let's save the data frame to a CSV file

In [18]:
dataframe_filtered.to_csv('VenuesRosedale.csv', index=False) 

#### Let's visualize these items on the map around our location


In [19]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Ecco


# add Rosedale as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Rosedale',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

<a ><img src = "https://github.com/kmiralles/Captstone-Data-Science/blob/main/RosedaleVenues.png?raw=true" > </a>

<a id="item4"></a>


## 4. Explore Trending Venues

> `https://api.foursquare.com/v2/venues/`**trending**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**


#### Now, instead of simply exploring the area around this location, we are interested in knowing the venues that are trending at this time, meaning the places with the highest foot traffic. So let's do that and get the trending venues around this location.


## Run the following lines at Breakfast time

In [20]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '600473af378c523e4a2d3acc'},
 'response': {'venues': []}}

### Check if any venues are trending at this time


In [21]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [22]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

## Save Trending Venues at Breakfast time to CSV

In [ ]:
trending_venues_df.df.to_csv('TrendingBreakfastRosedale.csv', index=False) 

Now, depending on when you run the above code, you might get different venues since the venues with the highest foot traffic are fetched live. 


### Visualize trending venues


In [ ]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [ ]:
# display map
venues_map

## Run the following lines at Lunch time

In [7]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '600cd1a702db8f06a1509693'},
 'response': {'venues': []}}

### Check if any venues are trending at this time


In [8]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [9]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

## Save Trending Venues at Lunch time to CSV

In [ ]:
trending_venues_df.df.to_csv('TrendingLunchRosedale.csv', index=False) 

Now, depending on when you run the above code, you might get different venues since the venues with the highest foot traffic are fetched live. 


### Visualize trending venues


In [ ]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [ ]:
# display map
venues_map

## Run the following lines at HappyHour time

In [20]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '600a10b99d9fea072c72da8a'},
 'response': {'venues': []}}

### Check if any venues are trending at this time


In [21]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [22]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

## Save Trending Venues at Happy Hour time to CSV

In [ ]:
trending_venues_df.df.to_csv('TrendingHappyHourRosedale.csv', index=False) 

Now, depending on when you run the above code, you might get different venues since the venues with the highest foot traffic are fetched live. 


### Visualize trending venues


In [ ]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [ ]:
# display map
venues_map

## Run the following lines at Dinner time

In [7]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '600a51b7c198684401f615fd'},
 'response': {'venues': []}}

### Check if any venues are trending at this time


In [8]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [9]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

## Save Trending Venues at Dinner time to CSV

In [ ]:
trending_venues_df.df.to_csv('TrendingDinnerRosedale.csv', index=False) 

Now, depending on when you run the above code, you might get different venues since the venues with the highest foot traffic are fetched live. 


### Visualize trending venues


In [ ]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [ ]:
# display map
venues_map

<a id="item6"></a>
